In [ ]:
docker run -it --entrypoint=bash python:3.12.8
pip --version

24.3.1


2 db:5432

## SQL

# CD to Docker_SQL

## start postgres in Docker
docker run -it \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v /workspaces/Data-Engineering-Zoomcamp/Docker_SQL/ny_taxi_postgres_data:/var/lib/postgresql/data \
  -p 5432:5432 \
  --network=pg-network \
  --name pg-database \
  postgres:13


## run the following to download the two data needed
python ./dataloader2.py     --user root     --password root     --host localhost     --port 5432     --db ny_taxi     --table-name yellow_taxi_data     --url "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"     --batch-size 10000     --max-workers 1
python ./dataloader2.py     --user root     --password root     --host localhost     --port 5432     --db ny_taxi     --table-name zones     --url "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"     --batch-size 10000     --max-workers 1


you can also use dockerfile





In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
user = "root"
password = "root"
host = "localhost"
port = "5432"
db = "ny_taxi"

In [4]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [5]:
engine.connect()

In [6]:
query = """
SELECT 1 as number;
"""

pd.read_sql(query, con=engine)

,number
0,1


In [7]:
query = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
"""

pd.read_sql(query, con=engine)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,yellow_taxi_data,root,None,False,False,False,False
1,public,zones,root,None,False,False,False,False


In [8]:
query = """
SELECT * FROM yellow_taxi_data LIMIT 10
"""

pd.read_sql(query, con=engine)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1,112,196,1,5.88,18,0.50,0.5,0,0,None,0.3,19.30,2,1,0
1,1,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1,43,263,1,0.80,5,3.25,0.5,0,0,None,0.3,9.05,2,1,0
2,1,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1,255,228,2,7.50,22,0.50,0.5,0,0,None,0.3,22.80,2,1,0
3,1,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1,181,181,1,0.90,6,0.50,0.5,0,0,None,0.3,6.80,2,1,0
4,2,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1,97,188,1,2.52,10,0.50,0.5,2,0,None,0.3,13.56,1,1,0
5,2,2019-10-01 00:35:01,2019-10-01 00:43:40,N,1,65,49,1,1.47,8,0.50,0.5,2,0,None,0.3,11.16,1,1,0
6,1,2019-10-01 00:28:09,2019-10-01 00:30:49,N,1,7,179,1,0.60,4,0.50,0.5,1,0,None,0.3,6.30,1,1,0
7,2,2019-10-01 00:28:26,2019-10-01 00:32:01,N,1,41,74,1,0.56,5,0.50,0.5,0,0,None,0.3,5.80,2,1,0
8,2,2019-10-01 00:14:01,2019-10-01 00:26:16,N,1,255,49,1,2.42,11,0.50,0.5,0,0,None,0.3,11.80,2,1,0
9,1,2019-10-01 00:03:03,2019-10-01 00:17:13,Y,1,130,131,1,3.40,13,0.50,0.5,3,0,None,0.3,17.15,1,1,0


In [9]:
query = """
SELECT * FROM zones LIMIT 10
"""

pd.read_sql(query, con=engine)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


In [12]:
query = """
SELECT
  CAST(lpep_dropoff_datetime AS DATE) as "day",
  "DOLocationID"
FROM
  yellow_taxi_data t
WHERE
  CAST(lpep_dropoff_datetime AS DATE) >= '2019-10-01' 
  AND CAST(lpep_dropoff_datetime AS DATE) < '2019-11-01';
"""

pd.read_sql(query, con=engine)

,day,DOLocationID
0,2019-10-01,196
1,2019-10-01,263
2,2019-10-01,228
3,2019-10-01,181
4,2019-10-01,188
...,...,...
476191,2019-10-31,197
476192,2019-10-31,88
476193,2019-10-31,68
476194,2019-10-31,193


### Question 3 Ans:104,802; 198,924; 109,603; 27,678; 35,189 (B)

During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

#### 1. up to 1mile

In [13]:
query = """
SELECT 
  COUNT(*) as short_trips
FROM
  yellow_taxi_data t
WHERE
  CAST(t.lpep_dropoff_datetime AS DATE) >= '2019-10-01' 
  AND CAST(t.lpep_dropoff_datetime AS DATE) < '2019-11-01'
  AND t.trip_distance <= 1;
"""

pd.read_sql(query, con=engine)

,short_trips
0,104802


#### 2. (1 (exclusive) to 3 miles (inclusive) )

In [24]:
start_date = '2019-10-01'
end_date = '2019-11-01'
lower_bound = 1
upper_bound = 3

query = """
SELECT 
  COUNT(*) as trips_count
FROM
  yellow_taxi_data t
WHERE
  CAST(t.lpep_dropoff_datetime AS DATE) >= %(start_date)s
  AND CAST(t.lpep_dropoff_datetime AS DATE) < %(end_date)s
  AND t.trip_distance > %(lower_bound)s
  AND t.trip_distance <= %(upper_bound)s;
"""

params = {
    'start_date': start_date,
    'end_date': end_date,
    'lower_bound': lower_bound,
    'upper_bound': upper_bound
}

pd.read_sql(query, con=engine, params=params)

,trips_count
0,198924


#### 3. (3 (exclusive) to 7 miles (inclusive) )


In [25]:
start_date = '2019-10-01'
end_date = '2019-11-01'
lower_bound = 3
upper_bound = 7

query = """
SELECT 
  COUNT(*) as trips_count
FROM
  yellow_taxi_data t
WHERE
  CAST(t.lpep_dropoff_datetime AS DATE) >= %(start_date)s
  AND CAST(t.lpep_dropoff_datetime AS DATE) < %(end_date)s
  AND t.trip_distance > %(lower_bound)s
  AND t.trip_distance <= %(upper_bound)s;
"""

params = {
    'start_date': start_date,
    'end_date': end_date,
    'lower_bound': lower_bound,
    'upper_bound': upper_bound
}

pd.read_sql(query, con=engine, params=params)

,trips_count
0,109603


#### 4 (7 (exclusive) to 10 miles (inclusive) )

In [26]:
start_date = '2019-10-01'
end_date = '2019-11-01'
lower_bound = 7
upper_bound = 10

query = """
SELECT 
  COUNT(*) as trips_count
FROM
  yellow_taxi_data t
WHERE
  CAST(t.lpep_dropoff_datetime AS DATE) >= %(start_date)s
  AND CAST(t.lpep_dropoff_datetime AS DATE) < %(end_date)s
  AND t.trip_distance > %(lower_bound)s
  AND t.trip_distance <= %(upper_bound)s;
"""

params = {
    'start_date': start_date,
    'end_date': end_date,
    'lower_bound': lower_bound,
    'upper_bound': upper_bound
}

pd.read_sql(query, con=engine, params=params)

,trips_count
0,27678


#### 5. above 10 miles

In [27]:
query = """
SELECT 
  COUNT(*) as long_trips
FROM
  yellow_taxi_data t
WHERE
  CAST(t.lpep_dropoff_datetime AS DATE) >= '2019-10-01' 
  AND CAST(t.lpep_dropoff_datetime AS DATE) < '2019-11-01'
  AND t.trip_distance > 10; 
"""

pd.read_sql(query, con=engine)

,long_trips
0,35189


### Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

Tip: For every day, we only care about one single trip with the longest distance.

In [28]:
query = """
SELECT 
  t.lpep_pickup_datetime,
  t.trip_distance,
  t.trip_type
FROM
  yellow_taxi_data t
ORDER BY t.trip_distance DESC
LIMIT 1;
"""

pd.read_sql(query, con=engine)

,lpep_pickup_datetime,trip_distance,trip_type
0,2019-10-31 23:23:41,515.89,1


In [29]:
query = """
SELECT 
  t.lpep_pickup_datetime,
  t.trip_distance,
  t.trip_type
FROM
  yellow_taxi_data t
WHERE
  t.trip_distance = (SELECT MAX(trip_distance) FROM yellow_taxi_data)
  AND
  t.trip_type=1
"""

pd.read_sql(query, con=engine)

,lpep_pickup_datetime,trip_distance,trip_type
0,2019-10-31 23:23:41,515.89,1


#### Question 5. Three biggest pickup zones

Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

Consider only lpep_pickup_datetime when filtering by date.

In [49]:
#cast as date to select date 

query = """
SELECT 
    CAST(t.lpep_pickup_datetime AS DATE) AS pickup_time,
    CAST(t.lpep_dropoff_datetime AS DATE) AS dropoff_time,
    SUM(total_amount),
    pickup."Zone" As pickup_zone,
    dropoff."Zone" As dropoff_zone,
    CONCAT(pickup."Borough", '/', pickup."Zone") AS pickup_loc,
    CONCAT(dropoff."Borough", '/', dropoff."Zone") AS dropoff_loc
FROM yellow_taxi_data t
JOIN zones pickup ON t."PULocationID" = pickup."LocationID"
JOIN zones dropoff ON t."DOLocationID" = dropoff."LocationID"
WHERE
    CAST(t.lpep_pickup_datetime AS DATE)='2019-10-18'
GROUP BY
    CAST(t.lpep_pickup_datetime AS DATE)
LIMIT 100
"""

pd.read_sql(query, con=engine)

,pickup_time,dropoff_time,total_amount,pickup_zone,dropoff_zone,pickup_loc,dropoff_loc
0,2019-10-18,2019-10-19,22.30,West Concourse,Clinton East,Bronx/West Concourse,Manhattan/Clinton East
1,2019-10-18,2019-10-18,8.16,Long Island City/Hunters Point,Greenpoint,Queens/Long Island City/Hunters Point,Brooklyn/Greenpoint
2,2019-10-18,2019-10-18,14.30,Forest Hills,South Jamaica,Queens/Forest Hills,Queens/South Jamaica
3,2019-10-18,2019-10-18,9.30,Elmhurst,Elmhurst,Queens/Elmhurst,Queens/Elmhurst
4,2019-10-18,2019-10-19,39.36,East Harlem North,Hamilton Heights,Manhattan/East Harlem North,Manhattan/Hamilton Heights
...,...,...,...,...,...,...,...
95,2019-10-18,2019-10-18,45.92,Kew Gardens,Co-Op City,Queens/Kew Gardens,Bronx/Co-Op City
96,2019-10-18,2019-10-18,41.30,West Concourse,Gramercy,Bronx/West Concourse,Manhattan/Gramercy
97,2019-10-18,2019-10-18,31.50,Highbridge,Astoria,Bronx/Highbridge,Queens/Astoria
98,2019-10-18,2019-10-18,34.26,West Concourse,Times Sq/Theatre District,Bronx/West Concourse,Manhattan/Times Sq/Theatre District


## the real answer is below (A)

In [ ]:
#cast as date to select date 
query = """
SELECT 
    pickup."Zone" AS pickup_zone,
    SUM(total_amount) AS total_amount_sum
FROM yellow_taxi_data t
JOIN zones pickup ON t."PULocationID" = pickup."LocationID"
JOIN zones dropoff ON t."DOLocationID" = dropoff."LocationID"
WHERE
    CAST(t.lpep_pickup_datetime AS DATE) = '2019-10-18'
GROUP BY
    pickup."Zone"
HAVING
    SUM(total_amount) > 13000
ORDER BY
    SUM(total_amount) DESC
LIMIT 100
"""

pd.read_sql(query, con=engine)

,pickup_zone,total_amount_sum
0,East Harlem North,18686.68
1,East Harlem South,16797.26
2,Morningside Heights,13029.79


### Question 6. Largest tip
For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

Note: it's tip , not trip

In [64]:
#cast as date to select date 

query = """
SELECT 
    dropoff."Zone" As dropoff_zone,
    MAX(t.tip_amount) AS largest_tip
FROM yellow_taxi_data t
JOIN zones pickup ON t."PULocationID" = pickup."LocationID"
JOIN zones dropoff ON t."DOLocationID" = dropoff."LocationID"
WHERE
    CAST(t.lpep_pickup_datetime AS DATE) >= '2019-10-01'
    AND 
    CAST(t.lpep_pickup_datetime AS DATE) < '2019-11-01'
    AND
    pickup."Zone" = 'East Harlem North'
GROUP BY
    dropoff_zone
ORDER BY largest_tip DESC
LIMIT 100
"""

pd.read_sql(query, con=engine)

,dropoff_zone,largest_tip
0,JFK Airport,87
1,Yorkville West,81
2,East Harlem North,40
3,Newark Airport,26
4,Upper East Side North,18
...,...,...
95,Garment District,7
96,Little Italy/NoLiTa,7
97,Kips Bay,7
98,Forest Hills,7


## Question 7. Terraform Workflow
Which of the following sequences, respectively, describes the workflow for:

Downloading the provider plugins and setting up backend, (terraform init)
Generating proposed changes and auto-executing the plan (terraform apply)
Remove all resources managed by terraform` (terraform destroy)